👋 In this kernel I will go through one possible approach, a histogram based method, to estimate breast densities. The method has been presented in Wu et al. (2018) \[1\] (see also Karssemeijer 1998 \[2\]). Wu et al. implement the histogram as a baseline method to compare agaist their convolutional neural network-based method.

<div class="alert alert-block alert-info">
<b>Tip:</b> The Breast Imaging Reporting and Data System (BI-RADS) is used to classify breast density into four categories: <b>"A"</b>: almost entirely fatty breast tissue, <b>"B"</b>: areas of dense glandular tissue and fibrous connective tissue, <b>"C"</b>: heterogeneously dense breast tissue with many areas of glandular tissue and fibrous connective tissue, <b>"D"</b>: extremely dense breast tissue.
</div>

<div class="alert alert-block alert-success">
    <b>Data source:</b> Radiological Society of North America. (2022, Nov 29). RSNA Screening Mammography Breast Cancer Detection, Version 1. Retrieved 2023 Feb 9 from [https://www.kaggle.com/competitions/rsna-breast-cancer-detection/data].
</div>

# Imports

In [ ]:
import os
import sys
import glob
import pathlib

import pandas as pd
import numpy as np

import pydicom
from pydicom.errors import InvalidDicomError
from pydicom.pixel_data_handlers.util import apply_voi_lut

import matplotlib.pyplot as plt

import torch

# Clone `breast_density_classifier` repository

The `breast_density_classifier` is published under BSD-2-Clause license which is compatible with the Apache 2.0 open source license under which this notebook has been made public.

In [ ]:
if not os.path.isdir('/kaggle/working/breast_density_classifier'):
    !git clone https://github.com/nyukat/breast_density_classifier.git

## Add path

In [ ]:
density_classifier_loc = '/kaggle/working/breast_density_classifier'
sys.path.append(density_classifier_loc)  # According to https://www.kaggle.com/general/135988#1985330

## Add `__init__.py`

Let's include `__init__.py` to access the files as modules.

In [ ]:
save_path = density_classifier_loc + '__init__.py'
if not os.path.isfile(save_path):
    open(save_path, 'a').close()

# Install dependencies

In addition to the commands below, there is also a wheel (https://www.kaggle.com/code/vslaykovsky/rsna-2022-whl) which can be used in an offline setting.

In [ ]:
!pip install python-gdcm -q
!pip install pylibjpeg -q

# Read metadata

In [ ]:
train_meta = pd.read_csv("/kaggle/input/rsna-breast-cancer-detection/train.csv")
train_meta.head()

In [ ]:
train_meta.loc[train_meta['density'] == 'C'].head()

In [ ]:
patient_id = '10038'  # folder name, corresponds to patient_id

# Read DICOM

In [ ]:
dcm_loc = pathlib.Path('/kaggle/input/rsna-breast-cancer-detection/train_images/')

In [ ]:
filelist = list(dcm_loc.glob(f'{patient_id}/*.dcm'))  # manually chosen patient_id from the table above
filelist

In [ ]:
# Source: https://www.kaggle.com/code/anttiisosalo/solt-based-image-augs-rsna-bc-detection

image_dict = {}
for i, file in enumerate(filelist):
    try:
        ds = pydicom.dcmread(str(file))
    except InvalidDicomError:
        print(f'Invalid DICOM file: {str(file)}')
        continue
    
    # print(ds)
    
    if hasattr(ds, 'SOPClassUID') and not ds.SOPClassUID == '1.2.840.10008.5.1.4.1.1.1.2':
        print(f'File has unknown SOPClassUID: {ds.SOPClassUID}')
        continue
    
    if hasattr(ds, 'PresentationIntentType') and not ds.PresentationIntentType == 'FOR PROCESSING':
        print(f'Invalid Presentation Intent Type: {ds.PresentationIntentType}')
        continue

    try:
        arr = ds.pixel_array.copy()  # get pixel array
    except AttributeError:
        print(f'AttributeError: Unable to return pixel array.')
        continue
    except ValueError:
        print(f'ValueError: Unable to return pixel array.')
        continue

    try:
        out = apply_voi_lut(arr, ds)  # apply VOI LUT (default LUT) or windowing operation
    except AttributeError:
        print(f'AttributeError: Unable to apply VOI LUT or windowing.')
        continue
    
    if hasattr(ds, 'PhotometricInterpretation') and not (ds.PhotometricInterpretation == 'MONOCHROME1' or ds.PhotometricInterpretation == 'MONOCHROME2'):
        print(f'Unknown photometric interpretation: {ds.PhotometricInterpretation}')
        continue
    elif not hasattr(ds, 'PhotometricInterpretation'):
        print(f'Missing photometric interpretation.')
        continue

    if ds.PhotometricInterpretation == 'MONOCHROME1':  # ranges from bright to dark with ascending pixel values
        out = out.max() - out
    elif ds.PhotometricInterpretation == 'MONOCHROME2':  # ranges from dark to bright with ascending pixel values
        pass

    height = ds.Rows
    width = ds.Columns
    
    out = out.reshape((height, width))  # (height, width)

    if hasattr(ds, 'ImageLaterality'):
        laterality = ds.ImageLaterality
    else:
        print(f'Unknown or invalid Laterality.')
        continue
    
    patient = ds.PatientID
    
    try:
        view = train_meta.loc[( train_meta['patient_id'] == int(patient) ) & ( train_meta['image_id'] == int(file.stem) ), 'view'].item()
    except ValueError:
        print(f'ValueError: Unable to return View.')
        continue

    bitdepth = ds.BitsAllocated
    # bitsstored = ds.BitsStored

    out = out.astype(np.float64)
    out /= out.max()
    out *= pow(2, bitdepth) - 1
    out = out.astype(np.uint16)

    if laterality == 'R' and view == 'MLO':
        image_dict['R-MLO'] = out
    elif laterality == 'L' and view == 'MLO':
        image_dict['L-MLO'] = out
    elif laterality == 'R' and view == 'CC':
        image_dict['R-CC'] = out
    elif laterality == 'L' and view == 'CC':
        image_dict['L-CC'] = out
    
    # study = ds.StudyInstanceUID

In [ ]:
image_dict

## Flip 'R' side images

In [ ]:
view_l_cc = image_dict['L-CC']
view_r_cc = np.fliplr(image_dict['R-CC'])
view_l_mlo = image_dict['L-MLO']
view_r_mlo = np.fliplr(image_dict['R-MLO'])

In [ ]:
for standard_view in [view_r_mlo, view_l_mlo, view_r_cc, view_l_cc]:
    fig = plt.figure(figsize=(7, 7))
    ax = fig.add_subplot(1,1,1)
    ax.imshow(standard_view, cmap='gray')
    plt.show()
    
    unique_colors = len(np.unique(standard_view))
    mi = np.min(standard_view)
    ma = np.max(standard_view)
    print(f'Unique colors: {unique_colors}; Min value: {mi}; Max value: {ma}')

# Crop images

Here we make use [´breast_segment´](https://github.com/olieidel/breast_segment/) script which is published under MIT license which is compatible with the Apache 2.0 open source license under which this notebook has been made public.

In [ ]:
#%%capture
"""
MIT License

Copyright (c) 2016 Oliver Eidel

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
""";

In [ ]:
from skimage.exposure import equalize_hist
from skimage.filters.rank import median
from skimage.measure import regionprops
from skimage.morphology import disk
from skimage.segmentation import felzenszwalb
from skimage.transform import rescale, resize

from scipy.ndimage import binary_fill_holes

from PIL import Image

In [ ]:
def get_crop_mask(im, scale_factor=0.25, threshold=3900, felzenzwalb_scale=0.15, flipped=True):
    im_thres = im.copy()
    im_thres[im_thres > threshold] = 0
    
    im_small = rescale(im_thres, scale_factor)
    im_small_filt = median(im_small, disk(50))
    
    im_small_filt = equalize_hist(im_small_filt)
    
    segments = felzenszwalb(im_small_filt, scale=felzenzwalb_scale)
    segments += 1  # to prevent labels() from ignoring segment with 0's
    
    props = regionprops(segments)
    props_sorted = sorted(props, key=lambda x: x.area, reverse=True)
    
    bg_index = 0
    
    bg_region = props_sorted[bg_index]
    minr, minc, maxr, maxc = bg_region.bbox
    filled_mask = bg_region.filled_image
    
    im_small_fill = np.zeros((im_small_filt.shape[0]+2, im_small_filt.shape[1]+1), dtype=int)
    
    im_small_fill[minr+1:maxr+1, minc:maxc] = filled_mask
    im_small_fill[0, :] = 1  # top
    im_small_fill[-1, :] = 1  # bottom
    im_small_fill[:, -1] = 1  # right
    
    im_small_fill = binary_fill_holes(im_small_fill)
    im_small_mask = im_small_fill[1:-1, :-1] if flipped == True else im_small_fill[1:-1, 1:]
    
    shape = (im.shape[1],im.shape[0])
    im_mask = np.array(Image.fromarray(im_small_mask).resize(shape)).astype(bool)
    
    im_mask = ~im_mask  # invert mask
    
    # determine side of breast in mask and compare
    col_sums_split = np.array_split(np.sum(im_mask, axis=0), 2)
    left_col_sum = np.sum(col_sums_split[0])
    right_col_sum = np.sum(col_sums_split[1])

    if left_col_sum > right_col_sum:
        breast_side_mask = True
    else:
        breast_side_mask = False

    if breast_side_mask != flipped:  # breast mask is not on the expected side
        im_mask = ~im_mask
    
    if im_mask.ravel().sum() == 0:  # if no region is found, abort and return mask the same size as the input image
        res = np.ones_like(im).astype(bool)
    else:
        minr = np.argwhere(im_mask.any(axis=1)).ravel()[0]
        maxr = np.argwhere(im_mask.any(axis=1)).ravel()[-1]
        minc = np.argwhere(im_mask.any(axis=0)).ravel()[0]
        maxc = np.argwhere(im_mask.any(axis=0)).ravel()[-1]
        res = im[minr:maxr, minc:maxc].copy()
    
    del im
    del im_thres
    del im_mask
    
    return res

In [ ]:
cropped_view_l_cc = get_crop_mask(view_l_cc, scale_factor=0.25, threshold=30000, felzenzwalb_scale=0.15, flipped=True)

In [ ]:
cropped_view_r_cc = get_crop_mask(view_r_cc, scale_factor=0.25, threshold=30000, felzenzwalb_scale=0.15, flipped=True)

In [ ]:
cropped_view_l_mlo = get_crop_mask(view_l_mlo, scale_factor=0.25, threshold=30000, felzenzwalb_scale=0.15, flipped=True)

In [ ]:
cropped_view_r_mlo = get_crop_mask(view_r_mlo, scale_factor=0.25, threshold=30000, felzenzwalb_scale=0.15, flipped=True)

In [ ]:
import torch.nn.functional as F

In [ ]:
new_shape = tuple(map(max, zip(*[cropped_view_l_cc.shape[:2], cropped_view_r_cc.shape[:2], cropped_view_l_mlo.shape[:2], cropped_view_r_mlo.shape[:2]])))
new_shape

In [ ]:
padded_view_l_cc = np.pad(cropped_view_l_cc.copy(), ((0, new_shape[0] - cropped_view_l_cc.shape[0]), (0, new_shape[1] - cropped_view_l_cc.shape[1])), constant_values=0)
padded_view_r_cc = np.pad(cropped_view_r_cc.copy(), ((0, new_shape[0] - cropped_view_r_cc.shape[0]), (0, new_shape[1] - cropped_view_r_cc.shape[1])), constant_values=0)
padded_view_l_mlo = np.pad(cropped_view_l_mlo.copy(), ((0, new_shape[0] - cropped_view_l_mlo.shape[0]), (0, new_shape[1] - cropped_view_l_mlo.shape[1])), constant_values=0)
padded_view_r_mlo = np.pad(cropped_view_r_mlo.copy(), ((0, new_shape[0] - cropped_view_r_mlo.shape[0]), (0, new_shape[1] - cropped_view_r_mlo.shape[1])), constant_values=0)
padded_view_l_cc.shape, padded_view_r_cc.shape, padded_view_l_mlo.shape, padded_view_r_mlo.shape

In [ ]:
for standard_view in [padded_view_r_mlo, padded_view_l_mlo, padded_view_r_cc, padded_view_l_cc]:
    fig = plt.figure(figsize=(7, 7))
    ax = fig.add_subplot(1,1,1)
    ax.imshow(standard_view, cmap='gray')
    plt.show()
    
    unique_colors = len(np.unique(standard_view))
    mi = np.min(standard_view)
    ma = np.max(standard_view)
    print(f'Unique colors: {unique_colors}; Min value: {mi}; Max value: {ma}')

# Density estimation

In [ ]:
#%%capture
"""
BSD 2-Clause License

Copyright (c) 2018, Nan Wu, Krzysztof J. Geras, Yiqiu Shen, Jingyi Su, 
S. Gene Kim, Eric Kim, Stacey Wolfson, Linda Moy, Kyunghyun Cho
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

* Redistributions of source code must retain the above copyright notice, this
  list of conditions and the following disclaimer.

* Redistributions in binary form must reproduce the above copyright notice,
  this list of conditions and the following disclaimer in the documentation
  and/or other materials provided with the distribution.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
""";

In [ ]:
import models_torch

In [ ]:
device = 'cpu'

In [ ]:
parameters = {'bins_histogram': 50}
model = models_torch.BaselineHistogramModel(num_bins=parameters['bins_histogram']).to(device)

In [ ]:
model_path = 'breast_density_classifier/saved_models/BreastDensity_BaselineHistogramModel/model.p'
model.load_state_dict(torch.load(model_path))

## Run model

In [ ]:
import utils

In [ ]:
x = torch.Tensor(utils.histogram_features_generator([padded_view_l_cc, padded_view_r_cc, padded_view_l_mlo, padded_view_r_mlo], parameters)).to(device)

In [ ]:
with torch.no_grad():
    predicted_density = model(x).cpu().numpy()

In [ ]:
print(f'A: {str(predicted_density[0, 0])}; ' \
      f'B: {str(predicted_density[0, 1])}; ' \
      f'C: {str(predicted_density[0, 2])}; ' \
      f'D: {str(predicted_density[0, 3])}; ')

# Combine results

In [ ]:
res = np.argmax([predicted_density[0, 0], predicted_density[0, 1], predicted_density[0, 2], predicted_density[0, 3]])

In [ ]:
density_classes = list('ABCD')

In [ ]:
np.array(density_classes)[res]

The histogram based result is highly dependent on the pre-processing, **cropping** and **number of bins**. The reference class for this patient is **'C'**. It should be noted, that density assesment is somewaht subjective and can also include observer variability.

# Conclusions

The predicted densities, when they are tuned to match the reference densities, can be used, for example, to partition the competition data according to the breast tissue density. 👍

# References

\[1\] Nan Wu, Krzysztof J. Geras, Yiqiu Shen, Jingyi Su, S. Gene Kim, Eric Kim, Stacey Wolfson, Linda Moy & Kyunghyun Cho, "Breast density classification with deep convolutional neural networks," In 2018 IEEE International Conference on Acoustics, Speech and Signal Processing (ICASSP), pp. 6682--6686, 2018. URL: https://github.com/nyukat/breast_density_classifier/

\[2\] Nico Karssemeijer, “Automated classification of parenchymal patterns in mammograms,” Physics in Medicine & Biology, vol. 43, no. 2, pp. 365--378, 1998. DOI: 10.1088/0031-9155/43/2/011.